# ETL Users Items

Importamos las librerias que utilizaremos en esta etapa del proyecto.

In [1]:
import pandas as pd
import numpy as np
import ast
import json

## Extracción de Datos:

Hemos comenzado descomprimiendo los archivos json.gz con WinRAR, haciendo que los datos estén listos para su procesamiento en VS Code. Luego lo cargo en un DataFrame para su uso posterior.

In [2]:
# Ruta al archivo JSON
raw_data = 'australian_users_items.json'

rows = []

# Probar con una codificación específica si utf-8 no funciona
with open(raw_data, encoding='MacRoman') as f:
    for line in f.readlines():
        try:
            rows.append(ast.literal_eval(line))
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_users_items = pd.DataFrame(rows)
print(df_users_items.head())


             user_id  items_count           steam_id  \
0  76561197970982479          277  76561197970982479   
1            js41637          888  76561198035864385   
2          evcentric          137  76561198007712555   
3         Riot-Punch          328  76561197963445855   
4              doctr          541  76561198002099482   

                                            user_url  \
0  http://steamcommunity.com/profiles/76561197970...   
1               http://steamcommunity.com/id/js41637   
2             http://steamcommunity.com/id/evcentric   
3            http://steamcommunity.com/id/Riot-Punch   
4                 http://steamcommunity.com/id/doctr   

                                               items  
0  [{'item_id': '10', 'item_name': 'Counter-Strik...  
1  [{'item_id': '10', 'item_name': 'Counter-Strik...  
2  [{'item_id': '1200', 'item_name': 'Red Orchest...  
3  [{'item_id': '10', 'item_name': 'Counter-Strik...  
4  [{'item_id': '300', 'item_name': 'Day of Defea..

In [3]:
df_users_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


## Transformación de Datos:

En segundo lugar, continuamos con la transformación. Esta consiste en limpiar, modificar y preparar los datos para su uso posterior. Eso incluye eliminar datos nulos, convertir tipos de datos, normalizar columnas, desanidar listas, etc.

### DATOS NULOS y COLUMNAS IRRELEVANTES

Ahora que se la magnitud del dataset empiezo a depurarlo. En primer lugar voy a cheuqear si hay o no **id** que tengan **valor nulo**, dado que eso me podria traer problemas al relacionar mis datos con los otros dataset. De existir **id** sin valor, entonces eliminaria esas filas.

Dada la información que entrega df_users_items.info() notamos que en este caso el DataFrame original  NO tiene valores nulos dado que la cantidad de filas se corresponden con la cantidad  de user_id, items_count, steam_id, user_url e items. Por ende no se justifica averiguar filas con data nula ni rearmarla. Por esta razón pasamos directamente a la eliminación de columnas que puedan estar de más para nuesstro objetivo puntual del proyecto. 

In [4]:
columns_relevant = ['user_id', 'items_count', 'items']
df_users_items2 = df_users_items[columns_relevant]
df_users_items2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   items        88310 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


### CALIDAD DE DATOS

In [5]:
df_users_items2.dtypes

user_id        object
items_count     int64
items          object
dtype: object

**user_id**: analizamos que tipos de datos tiene dado que está definido como object. AL ver tan solo algunos ejemplos de la tabla notamos que hay tanto nombres como números. Por ello no se justifica modifcarlos.

In [6]:
df_users_items2.head(10)

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,371,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
6,NitemarePK,304,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
7,themanwich,258,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
8,maplemage,629,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
9,Wackky,0,[]


**items**: En este caso, lo primero y más importante será desanidar los datos, dado que como se observa en la tabla de arriba estan en formato JSON anidados en en cada dato de la columna.

In [7]:
def items_extract(df):
    items_list = []
    
    # Iterar sobre cada fila del DataFrame original
    for _, row in df.iterrows():
        user_id = row['user_id']
        items_count = row['items_count']
        items = row['items']
        
        # Iterar sobre cada item en la lista de items para extraer la información necesaria
        for item in items:
            item_id = item.get('item_id')
            item_name = item.get('item_name')
            playtime_forever = item.get('playtime_forever')
            
            items_list.append([user_id, items_count, item_id, item_name, playtime_forever])
    
    # Crear un nuevo DataFrame
    df_items = pd.DataFrame(items_list, columns=['user_id', 'items_count', 'item_id', 'item_name', 'playtime_forever'])
    return df_items

# Llamar a la función para extraer los datos de items
df_users_items3 = items_extract(df_users_items2)

# Ver las primeras filas del nuevo DataFrame
print(df_users_items3.head())


             user_id  items_count item_id                  item_name  \
0  76561197970982479          277      10             Counter-Strike   
1  76561197970982479          277      20      Team Fortress Classic   
2  76561197970982479          277      30              Day of Defeat   
3  76561197970982479          277      40         Deathmatch Classic   
4  76561197970982479          277      50  Half-Life: Opposing Force   

   playtime_forever  
0                 6  
1                 0  
2                 7  
3                 0  
4                 0  


In [8]:
df_users_items3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   items_count       int64 
 2   item_id           object
 3   item_name         object
 4   playtime_forever  int64 
dtypes: int64(2), object(3)
memory usage: 196.6+ MB


In [9]:

# Convertir tanto item_id como playtime_forever a enteros
df_users_items3['item_id'] = df_users_items3['item_id'].astype(int)
df_users_items3['playtime_forever'] = df_users_items3['playtime_forever'].astype(float)

# Verificar los tipos de datos
print(df_users_items3.dtypes)

user_id              object
items_count           int64
item_id               int64
item_name            object
playtime_forever    float64
dtype: object


### Segunda etapa de depuración del DataFrame

Dado que desanidamos la columna item, nuestro dataframe paso de tener 88.310 filas a 5.153.209. Por ende el tamaño de nuestra base creceria considerablemente y ello justifica volver a chequear datos nulos, duplicados, etc.

In [10]:
# Registros con id nulo: 
null_register = df_users_items3.isna().sum().sum()
null_register


np.int64(0)

Dado que el tamaño del dataframe sigue siendo muy grande, y podria traerme consecuencias negativas para las siguientes etapas, procederé a una limpieza mayor.

**Eliminacion de mínimos** 

Dado que parte importante del uso de este dataset sera para calcular el tiempo máximo de juego segun genero, consideramos que podriamos filtrar la cantidad de filas en base a aquellas en donde el tiempo de juego es menor a una hora.

In [11]:
df_users_items4 = df_users_items3[df_users_items3['playtime_forever'] >= 1]
df_users_items4.info()


<class 'pandas.core.frame.DataFrame'>
Index: 3285246 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   item_id           int64  
 3   item_name         object 
 4   playtime_forever  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 150.4+ MB


Como vemos se redujo el numero de filas significativamente pero aun sigue siendo un valor grande que puede traer problemas a la hora de querer trabajar con el en fasapi y render. Así que evaluaremos cuanto tiempo jugo el percentil que menos jugó y de allí eliminaremos ese porcentaje tambien dado que no influiria en nuestros endpoints demasiado. Y luego en base a esa información procederemos a achicar nuevamente el dataset.

In [12]:
cuartil_50 = df_users_items4['playtime_forever'].quantile(0.50)
print(f"Cuartil 50 (Mediana): {cuartil_50}")


Cuartil 50 (Mediana): 205.0


In [13]:
df_users_items5 = df_users_items4[df_users_items4['playtime_forever'] > cuartil_50]
df_users_items5.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1641489 entries, 8 to 5153202
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   user_id           1641489 non-null  object 
 1   items_count       1641489 non-null  int64  
 2   item_id           1641489 non-null  int64  
 3   item_name         1641489 non-null  object 
 4   playtime_forever  1641489 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 75.1+ MB


Dado que aún se trata de un dataset muy grande que podria traer conflicto en los siguentes niveles, realizo una muestra significativa aleatoria. Elijo que sea aleatoria y no de los primero valores para que así sea más representativa de toda la muestra.

In [14]:
# Crear una muestra aleatoria de 50,000 registros
df_users_items6 = df_users_items5.sample(n=50000, random_state=27)
df_users_items6.head()

,user_id,items_count,item_id,item_name,playtime_forever
2589777,76561198070585472,4456,340330,Deity Quest,323.0
2697140,Hyren516,63,211820,Starbound,403.0
1916843,64723864,221,346900,AdVenture Capitalist,1374.0
3104352,nerothefgt,183,550,Left 4 Dead 2,1424.0
2902020,76561198015803033,127,4000,Garry's Mod,9797.0


### Cuidado Estético y Reorganización de columnas

In [15]:
# Normalizar nombres de columnas a español
new_column_names = {
    'user_id': 'id_usuario',
    'item_count': 'cantidad_de_items',
    'item_id': 'id_item',
    'item_name': 'nombre_de_item',
    'playtime_forever': 'tiempo_total_de_juego'
}
df_users_items6 = df_users_items6.rename(columns=new_column_names)
df_users_items6.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 2589777 to 3446428
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id_usuario             50000 non-null  object 
 1   items_count            50000 non-null  int64  
 2   id_item                50000 non-null  int64  
 3   nombre_de_item         50000 non-null  object 
 4   tiempo_total_de_juego  50000 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.3+ MB


In [16]:
df_users_items6.head(10)

,id_usuario,items_count,id_item,nombre_de_item,tiempo_total_de_juego
2589777,76561198070585472,4456,340330,Deity Quest,323.0
2697140,Hyren516,63,211820,Starbound,403.0
1916843,64723864,221,346900,AdVenture Capitalist,1374.0
3104352,nerothefgt,183,550,Left 4 Dead 2,1424.0
2902020,76561198015803033,127,4000,Garry's Mod,9797.0
2384564,Bindalee,28,238090,Sniper Elite 3,1008.0
1502539,76561198131938966,60,33930,Arma 2: Operation Arrowhead,11499.0
1888957,JiAltair,45,4000,Garry's Mod,412.0
2606353,Ihealwithmypenis,114,108600,Project Zomboid,417.0
1619212,76561198071322532,55,200260,Batman: Arkham City GOTY,1661.0


## Carga de Datos (Load)

En esta fase final del proceso ETL, los datos limpios y transformados se cargan en un sistema de almacenamiento o base de datos de destino. En nuestro caso utilizaremos Parquet dada su eficiencia. Y quedan ya listos para ser usados por FastAPI y Render.

In [17]:
import pyarrow
import fastparquet

In [18]:
# Guardar el DataFrame en formato CSV
df_users_items6.to_csv('australian_users_items.csv', index=False)
print("Archivo CSV guardado exitosamente.")

# Guardar el DataFrame en formato Parquet
df_users_items6.to_parquet('australian_users_items.parquet', index=False, engine='pyarrow', compression='snappy')
print("Archivo Parquet guardado exitosamente.")

Archivo CSV guardado exitosamente.
Archivo Parquet guardado exitosamente.


In [20]:
dfpr = pd.read_parquet('australian_users_items.parquet')

# Mostrar los nombres de las columnas
print(dfpr.columns)

Index(['id_usuario', 'items_count', 'id_item', 'nombre_de_item',
       'tiempo_total_de_juego'],
      dtype='object')
